In [ ]:
import librosa
import numpy as np

import torch
import torch.nn as nn
import torchaudio

from vector_quantize_pytorch import ResidualVQ
from conformer import Conformer

In [ ]:
wav_path = './rvq_input.wav'
wav, sr = torchaudio.load(wav_path)
mel = librosa.feature.melspectrogram(
    y=wav.numpy(), sr=sr, n_fft=1024, hop_length=256, n_mels=80)
mel = librosa.power_to_db(mel, ref=np.max)
mel = torch.tensor(mel, dtype=torch.float)

In [ ]:
proj_linear = nn.Liear(80, 512)
posterior_encoder = Conformer(dim = 512, depth = 6, dim_head = 64,
                            heads = 8, ff_mult = 1024, conv_expansion_factor = 2,
                            conv_kernel_size = 31, attn_dropout = 0., ff_dropout = 0.,
                            conv_dropout = 0.)
decoder = Conformer(dim = 512, depth = 6, dim_head = 64,
                            heads = 8, ff_mult = 1024, conv_expansion_factor = 2,
                            conv_kernel_size = 31, attn_dropout = 0., ff_dropout = 0.,
                            conv_dropout = 0.)
mel_liner = nn.Linear(512, 80)


In [ ]:
residual_vq = ResidualVQ(
    dim = 512,
    num_quantizers = 8,      # specify number of quantizers
    codebook_size = 1024,    # codebook size
)

In [ ]:
feat = proj_linear(mel.transpose(1,2))
z = posterior_encoder(feat)
quantized, indices, commit_loss = residual_vq(z)
pred_mels = decoder(quantized)
pred_mels = mel_liner(pred_mels)


In [ ]:
# 만약 예측된 code가 있다면
z = residual_vq.get_output_from_indices(quantized)